In [76]:
!pip install python-louvain
!pip install ripser

In [101]:
#!/usr/bin/env python3
# generate_fgraph_features.py
# ---------------------------
# Usage examples:
#   python generate_fgraph_features.py --input fgraph_edges.csv --output fgraph_features.csv
#   python generate_fgraph_features.py --input fgraph_edges.csv --output "feat_{batch}.csv" --batch-size 100000
#
# CSV requirements:
#   - Must contain an 'EDGES' column with Python-like edge lists per row, e.g. "[(0,1),(1,2)]"
#   - If present, 'COEFFICIENTS' will be preserved in the output

import os, argparse, ast, warnings
from collections import Counter, defaultdict
from itertools import combinations
from math import log2
import math
import random

# Keep native libs from oversubscribing when we use parallelism
os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("OPENBLAS_NUM_THREADS", "1")
os.environ.setdefault("MKL_NUM_THREADS", "1")
os.environ.setdefault("BLIS_NUM_THREADS", "1")
os.environ.setdefault("VECLIB_MAXIMUM_THREADS", "1")
os.environ.setdefault("NUMEXPR_NUM_THREADS", "1")

# Silence noisy warnings you asked to ignore
warnings.filterwarnings("ignore", message="Precision loss occurred in moment calculation.*", category=RuntimeWarning)
try:
    # pandas SettingWithCopyWarning path
    import pandas as pd
    from pandas.errors import SettingWithCopyWarning
    warnings.filterwarnings("ignore", category=SettingWithCopyWarning)
except Exception:
    pass

import networkx as nx
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.stats import skew

# Optional parallel
try:
    from joblib import Parallel, delayed
    _HAS_JOBLIB = True
except Exception:
    _HAS_JOBLIB = False

# Optional joblib progress
try:
    from tqdm_joblib import tqdm_joblib
    _HAS_TQDM_JOBLIB = True
except Exception:
    _HAS_TQDM_JOBLIB = False

# Optional TDA (quickly skipped if not installed)
try:
    from ripser import ripser  # type: ignore
    _HAS_RIPSER = True
except Exception:
    _HAS_RIPSER = False

# ------------------------------ Small helpers ------------------------------ #
def shannon_entropy(counter: Counter) -> float:
    tot = sum(counter.values())
    return np.nan if tot == 0 else -sum((c / tot) * log2(c / tot) for c in counter.values())

def try_or_nan(func, *args, **kwargs):
    try:
        return func(*args, **kwargs)
    except Exception:
        return np.nan

def safe_comb(n, k):
    """Safe n choose k. Returns np.nan if n is None/NaN or n < k."""
    if n is None:
        return np.nan
    try:
        n = int(n)
        if n >= k:
            return math.comb(n, k)
        return np.nan
    except Exception:
        return np.nan

def gini(x):
    """Gini coefficient for a sequence."""
    x = np.asarray(list(x), dtype=float)
    x = x[np.isfinite(x)]
    if x.size == 0:
        return np.nan
    if np.all(x == 0):
        return 0.0
    x = np.sort(x)
    n = x.size
    cum = np.cumsum(x)
    return float((n + 1 - 2*np.sum(cum)/cum[-1]) / n)

def _q(vals, q):
    if not vals:
        return np.nan
    a = np.asarray(vals, dtype=float)
    a = a[np.isfinite(a)]
    return float(np.quantile(a, q)) if a.size else np.nan

def _frac(arr, pred):
    if not arr:
        return np.nan
    a = np.asarray(arr, dtype=float)
    a = a[np.isfinite(a)]
    if a.size == 0:
        return np.nan
    return float(np.mean(pred(a)))

def extract_faces(embedding: nx.PlanarEmbedding):
    """Return list of faces (as node-cycles) from a PlanarEmbedding."""
    seen = set()
    faces = []
    for u in embedding:
        for v in embedding[u]:
            if (u, v) in seen:
                continue
            face = embedding.traverse_face(u, v)
            faces.append(face)
            seen.update((face[i], face[(i+1)%len(face)]) for i in range(len(face)))
    return faces

def compute_node_orbits(graph):
    """Very simple (and expensive) automorphism orbit computation via GraphMatcher."""
    matcher = nx.algorithms.isomorphism.GraphMatcher(graph, graph)
    orbit_map = defaultdict(set)
    for iso in matcher.isomorphisms_iter():
        for u, v in iso.items():
            orbit_map[u].add(v)
    seen = set()
    orbits = []
    for group in orbit_map.values():
        g_frozen = frozenset(group)
        if g_frozen not in seen:
            seen.add(g_frozen)
            orbits.append(group)
    return orbits

# ------------------------------ Motif helpers ------------------------------ #
def _adjacency_matrix(G):
    return nx.to_numpy_array(G, dtype=float, nodelist=list(G.nodes()))

def _triangle_count(G, A=None):
    if A is None:
        A = _adjacency_matrix(G)
    A3 = A @ A @ A
    return int(round(np.trace(A3) / 6.0))

def _wedge_count(G, degs=None, tri=None):
    if degs is None:
        degs = [d for _, d in G.degree()]
    if tri is None:
        tri = _triangle_count(G)
    s = sum(d*(d-1)//2 for d in degs)
    return int(s - 3*tri)

def _four_cycle_count(G, A=None):
    if A is None:
        A = _adjacency_matrix(G)
    A2 = A @ A
    n = A2.shape[0]
    total_pairs = 0
    for i in range(n):
        for j in range(i+1, n):
            cij = int(round(A2[i, j]))
            if cij >= 2:
                total_pairs += (cij * (cij - 1)) // 2
    return int(total_pairs // 2)

def _four_clique_count(G):
    cnt = 0
    for clq in nx.find_cliques(G):
        k = len(clq)
        if k >= 4:
            cnt += math.comb(k, 4)
    return int(cnt)

def _edge_triangle_incidence_stats(G):
    out = []
    for u, v in G.edges():
        out.append(len(set(G[u]) & set(G[v])))
    if not out:
        return 0.0, 0.0
    arr = np.array(out, dtype=float)
    return float(arr.mean()), float(arr.std(ddof=0))

def _square_clustering_proxy(G, A=None):
    """
    Proxy for square clustering using 4-cycles.
    ratio = 4 * (#C4) / sum_{i<j} C( (A^2)_{ij}, 2 ), bounded in [0,1] when denom>0.
    """
    if A is None:
        A = _adjacency_matrix(G)
    A2 = A @ A
    n = A.shape[0]
    denom = 0
    for i in range(n):
        for j in range(i+1, n):
            cij = int(round(A2[i, j]))
            if cij >= 2:
                denom += (cij * (cij - 1)) // 2
    c4 = _four_cycle_count(G, A=A)
    if denom == 0:
        return np.nan
    return float(4.0 * c4 / denom)

# ---- Induced 4-node motif classification (connected graphlets only) ------- #
def _classify_induced4(G_sub):
    if not nx.is_connected(G_sub):
        return None
    m = G_sub.number_of_edges()
    degs = sorted([d for _, d in G_sub.degree()])
    if m == 3:
        if degs == [1,1,2,2]: return "P4"
        if degs == [1,1,1,3]: return "K1_3"
    elif m == 4:
        if degs == [2,2,2,2]: return "C4"
        if degs == [1,2,2,3]: return "TailedTriangle"
    elif m == 5:
        if degs == [2,2,3,3]: return "Diamond"
    elif m == 6:
        if degs == [3,3,3,3]: return "K4"
    return None

def _induced4_counts(G):
    counts = {
        "K1_3": 0, "P4": 0, "C4": 0,
        "TailedTriangle": 0, "Diamond": 0, "K4": 0
    }
    nodes = list(G.nodes())
    for S in combinations(nodes, 4):
        H = G.subgraph(S)
        key = _classify_induced4(H)
        if key is not None:
            counts[key] += 1
    return counts

# --------------------------- Spectral helpers ------------------------------ #
def _adjacency_spectrum_features(A):
    evals = np.linalg.eigvalsh(A)  # symmetric
    energy  = float(np.sum(np.abs(evals)))
    estrada = float(np.sum(np.exp(evals)))
    n = len(evals) if len(evals) > 0 else np.nan
    m2 = float(np.mean(evals**2)) if n == n and n > 0 else np.nan  # ~ 2m/n
    m3 = float(np.mean(evals**3)) if n == n and n > 0 else np.nan
    m4 = float(np.mean(evals**4)) if n == n and n > 0 else np.nan
    return energy, estrada, m2, m3, m4, evals

def _laplacian_heat_traces(leigs, ts=(0.1, 1.0, 5.0)):
    traces = {}
    if leigs is None or len(leigs) == 0 or np.any(np.isnan(leigs)):
        for t in ts:
            traces[f"Spectral_laplacian_heat_trace_t{t}"] = np.nan
        return traces
    for t in ts:
        traces[f"Spectral_laplacian_heat_trace_t{t}"] = float(np.sum(np.exp(-t * leigs)))
    return traces

# ----------------------------- TDA helpers --------------------------------- #
def _tda_vr_h_summary_from_spdm(G):
    """
    Vietoris–Rips PH on shortest-path distance matrices (component-wise).
    Returns dict with H0/H1 summaries and Betti at radii given by pairwise
    distance quantiles. If ripser not installed, returns NaN quickly.
    """
    out = {
        # H0
        "TDA_H0_count": np.nan, "TDA_H0_total_persistence": np.nan,
        "TDA_H0_mean_persistence": np.nan, "TDA_H0_max_persistence": np.nan,
        "TDA_H0_persistence_entropy": np.nan,
        "TDA_H0_mean_birth": np.nan, "TDA_H0_mean_death": np.nan,
        # H1
        "TDA_H1_count": np.nan, "TDA_H1_total_persistence": np.nan,
        "TDA_H1_mean_persistence": np.nan, "TDA_H1_max_persistence": np.nan,
        "TDA_H1_persistence_entropy": np.nan,
        "TDA_H1_mean_birth": np.nan, "TDA_H1_mean_death": np.nan,
        # Betti at quantiles
        "TDA_Betti0_at_q25": np.nan, "TDA_Betti0_at_q50": np.nan, "TDA_Betti0_at_q75": np.nan,
        "TDA_Betti1_at_q25": np.nan, "TDA_Betti1_at_q50": np.nan, "TDA_Betti1_at_q75": np.nan,
    }
    if not _HAS_RIPSER:
        return out

    comps = [G.subgraph(c).copy() for c in nx.connected_components(G)]
    if len(comps) == 0:
        return out

    # Collect distances for quantiles
    dists = []
    comp_dist = []
    for H in comps:
        nodes = list(H.nodes())
        if H.number_of_nodes() == 1:
            comp_dist.append(None)
            continue
        D = np.array(nx.floyd_warshall_numpy(H, nodelist=nodes), dtype=float)
        vals = D[~np.eye(D.shape[0], dtype=bool)]
        if vals.size > 0:
            dists.extend(vals.tolist())
        comp_dist.append((D, nodes))

    if len(dists) == 0:
        out.update({
            "TDA_H0_count": 0.0, "TDA_H0_total_persistence": 0.0,
            "TDA_H0_mean_persistence": 0.0, "TDA_H0_max_persistence": 0.0,
            "TDA_H0_persistence_entropy": 0.0,
            "TDA_H0_mean_birth": 0.0, "TDA_H0_mean_death": 0.0,
            "TDA_H1_count": 0.0, "TDA_H1_total_persistence": 0.0,
            "TDA_H1_mean_persistence": 0.0, "TDA_H1_max_persistence": 0.0,
            "TDA_H1_persistence_entropy": 0.0,
            "TDA_H1_mean_birth": 0.0, "TDA_H1_mean_death": 0.0,
            "TDA_Betti0_at_q25": 1.0, "TDA_Betti0_at_q50": 1.0, "TDA_Betti0_at_q75": 1.0,
            "TDA_Betti1_at_q25": 0.0, "TDA_Betti1_at_q50": 0.0, "TDA_Betti1_at_q75": 0.0,
        })
        return out

    q25, q50, q75 = np.quantile(np.array(dists), [0.25, 0.50, 0.75])

    def _agg_ph(dgm):
        finite = dgm[np.isfinite(dgm[:,1])] if dgm.size else np.empty((0,2))
        if finite.size == 0:
            return dict(count=0.0, total=0.0, mean=0.0, max_=0.0, ent=0.0,
                        mean_birth=0.0, mean_death=0.0)
        pers = finite[:,1] - finite[:,0]
        total = float(np.sum(pers))
        meanp = float(np.mean(pers))
        maxp  = float(np.max(pers))
        probs = pers / (total if total > 0 else 1.0)
        ent = -float(np.sum([p*np.log(p) for p in probs if p > 0]))
        return dict(count=float(finite.shape[0]), total=total, mean=meanp, max_=maxp, ent=ent,
                    mean_birth=float(np.mean(finite[:,0])), mean_death=float(np.mean(finite[:,1])))

    def _betti_at(dgm, t):
        if dgm.size == 0:
            return 0.0
        birth = dgm[:,0]; death = dgm[:,1]
        return float(np.sum((birth <= t) & (t < death)))

    H0_list, H1_list = [], []
    betti0_q = [0.0, 0.0, 0.0]
    betti1_q = [0.0, 0.0, 0.0]

    for item in comp_dist:
        if item is None:
            betti0_q = [b+1.0 for b in betti0_q]  # singleton component
            continue
        D, _ = item
        res = ripser(D, distance_matrix=True, maxdim=1)
        dgms = res.get("dgms", [])
        H0 = dgms[0] if len(dgms) > 0 else np.empty((0,2))
        H1 = dgms[1] if len(dgms) > 1 else np.empty((0,2))
        H0_list.append(_agg_ph(H0))
        H1_list.append(_agg_ph(H1))
        for idx, t in enumerate((q25, q50, q75)):
            betti0_q[idx] += _betti_at(H0, t)
            betti1_q[idx] += _betti_at(H1, t)

    def _combine(stats_list):
        if not stats_list:
            return dict(count=0.0, total=0.0, mean=0.0, max_=0.0, ent=0.0, mean_birth=0.0, mean_death=0.0)
        count = float(sum(s["count"] for s in stats_list))
        total = float(sum(s["total"] for s in stats_list))
        mean  = float(np.mean([s["mean"] for s in stats_list]))
        max_  = float(max([s["max_"] for s in stats_list]))
        ent   = float(np.mean([s["ent"] for s in stats_list]))
        mean_birth = float(np.mean([s["mean_birth"] for s in stats_list]))
        mean_death = float(np.mean([s["mean_death"] for s in stats_list]))
        return dict(count=count, total=total, mean=mean, max_=max_, ent=ent,
                    mean_birth=mean_birth, mean_death=mean_death)

    H0c = _combine(H0_list)
    H1c = _combine(H1_list)

    out.update({
        "TDA_H0_count": H0c["count"],
        "TDA_H0_total_persistence": H0c["total"],
        "TDA_H0_mean_persistence": H0c["mean"],
        "TDA_H0_max_persistence": H0c["max_"],
        "TDA_H0_persistence_entropy": H0c["ent"],
        "TDA_H0_mean_birth": H0c["mean_birth"],
        "TDA_H0_mean_death": H0c["mean_death"],
        "TDA_H1_count": H1c["count"],
        "TDA_H1_total_persistence": H1c["total"],
        "TDA_H1_mean_persistence": H1c["mean"],
        "TDA_H1_max_persistence": H1c["max_"],
        "TDA_H1_persistence_entropy": H1c["ent"],
        "TDA_H1_mean_birth": H1c["mean_birth"],
        "TDA_H1_mean_death": H1c["mean_death"],
        "TDA_Betti0_at_q25": betti0_q[0], "TDA_Betti0_at_q50": betti0_q[1], "TDA_Betti0_at_q75": betti0_q[2],
        "TDA_Betti1_at_q25": betti1_q[0], "TDA_Betti1_at_q50": betti1_q[1], "TDA_Betti1_at_q75": betti1_q[2],
    })
    return out

# -------------------- Extra coverage & distance/community ------------------- #
def _distance_stats_sampled(G, max_sources=200, exact_if_leq=500):
    """
    Approx distances: p90 shortest-path length (effective diameter),
    eccentricity mean and p90. Uses exact APSP if small & connected,
    otherwise samples up to max_sources BFS trees.
    """
    n = G.number_of_nodes()
    if n == 0:
        return np.nan, np.nan, np.nan
    try:
        if nx.is_connected(G) and n <= exact_if_leq:
            sp = dict(nx.all_pairs_shortest_path_length(G))
            dists = [d for u in sp for d in sp[u].values()]
            ecc   = list(nx.eccentricity(G).values())
        else:
            nodes = list(G.nodes())
            k = min(max_sources, len(nodes))
            seeds = random.sample(nodes, k) if k < len(nodes) else nodes
            dists, ecc = [], []
            for s in seeds:
                lengths = nx.single_source_shortest_path_length(G, s)
                if lengths:
                    dists.extend(lengths.values())
                    ecc.append(max(lengths.values()))
        d = np.asarray(dists, dtype=float)
        e = np.asarray(ecc, dtype=float)
        d = d[np.isfinite(d)]; e = e[np.isfinite(e)]
        eff_p90 = float(np.quantile(d, 0.90)) if d.size else np.nan
        ecc_mean = float(np.mean(e)) if e.size else np.nan
        ecc_p90  = float(np.quantile(e, 0.90)) if e.size else np.nan
        return eff_p90, ecc_mean, ecc_p90
    except Exception:
        return np.nan, np.nan, np.nan

def _community_features(G):
    """
    Louvain community features (if python-louvain installed).
    Returns dict with NaNs on failure/empty graph.
    """
    base = {
        "Comm_modularity": np.nan, "Comm_count": np.nan,
        "Comm_size_max": np.nan, "Comm_size_gini": np.nan,
        "Comm_internal_edge_frac": np.nan
    }
    try:
        import community as community_louvain  # python-louvain
        if G.number_of_edges() == 0 or G.number_of_nodes() == 0:
            return base
        part = community_louvain.best_partition(G)
        if not part:
            return base
        sizes = list(Counter(part.values()).values())
        internal = sum(1 for u, v in G.edges() if part.get(u) == part.get(v))
        base["Comm_modularity"] = float(community_louvain.modularity(part, G))
        base["Comm_count"] = float(len(sizes))
        base["Comm_size_max"] = float(max(sizes)) if sizes else np.nan
        base["Comm_size_gini"] = gini(sizes)
        base["Comm_internal_edge_frac"] = float(internal / G.number_of_edges())
        return base
    except Exception:
        return base

# ----------------------------- Feature extractor --------------------------- #
def extract_features(edge_list):
    G = nx.Graph()
    G.add_edges_from(edge_list)

    # full key list so empty graphs get consistent NaN rows
    base_keys = [
        # --- Original ---
        "Basic_num_nodes","Basic_num_edges","Basic_min_degree","Basic_max_degree",
        "Basic_avg_degree","Basic_degree_std","Basic_degree_skew","Basic_density",
        "Basic_edge_to_node_ratio","Basic_degree_entropy",
        "Connectivity_is_connected","Connectivity_num_components","Connectivity_diameter",
        "Connectivity_radius","Connectivity_avg_shortest_path_length","Connectivity_wiener_index",
        "Centrality_betweenness_mean","Centrality_betweenness_max","Centrality_betweenness_std","Centrality_betweenness_skew",
        "Centrality_closeness_mean","Centrality_closeness_max","Centrality_closeness_std","Centrality_closeness_skew",
        "Centrality_eigenvector_mean","Centrality_eigenvector_max","Centrality_eigenvector_std","Centrality_eigenvector_skew",
        "Core_max_core_index","Core_core_index_mean",
        "Robust_articulation_points","Robust_bridge_count",
        "Cycle_num_cycles_len_5","Cycle_num_cycles_len_6",
        "Spectral_algebraic_connectivity","Spectral_spectral_gap","Spectral_laplacian_mean",
        "Spectral_laplacian_std","Spectral_laplacian_skew","Kirchhoff_index",
        *[f"Spectral_lap_eig_{i}" for i in range(10)],
        "Planarity_num_faces","Planarity_face_size_mean","Planarity_face_size_max",
        "Symmetry_automorphism_group_order","Symmetry_num_orbits","Symmetry_orbit_size_max",

        # --- New motifs (raw) ---
        "Motif_triangles","Motif_wedges","Motif_4_cycles","Motif_4_cliques",
        "Motif_triangle_edge_incidence_mean","Motif_triangle_edge_incidence_std",
        "Motif_square_clustering_proxy",
        # Induced 4-node
        "Motif_induced_K1_3","Motif_induced_P4","Motif_induced_C4",
        "Motif_induced_TailedTriangle","Motif_induced_Diamond","Motif_induced_K4",
        "Motif_induced_connected_per_4set",

        # --- New spectral/energy (raw) ---
        "Adjacency_energy","Adjacency_estrada_index",
        "Adjacency_moment_2","Adjacency_moment_3","Adjacency_moment_4",
        "Spectral_laplacian_heat_trace_t0.1","Spectral_laplacian_heat_trace_t1.0","Spectral_laplacian_heat_trace_t5.0",

        # --- TDA (raw) ---
        "TDA_H0_count","TDA_H0_total_persistence","TDA_H0_mean_persistence","TDA_H0_max_persistence",
        "TDA_H0_persistence_entropy","TDA_H0_mean_birth","TDA_H0_mean_death",
        "TDA_H1_count","TDA_H1_total_persistence","TDA_H1_mean_persistence","TDA_H1_max_persistence",
        "TDA_H1_persistence_entropy","TDA_H1_mean_birth","TDA_H1_mean_death",
        "TDA_Betti0_at_q25","TDA_Betti0_at_q50","TDA_Betti0_at_q75",
        "TDA_Betti1_at_q25","TDA_Betti1_at_q50","TDA_Betti1_at_q75",

        # --- Normalized variants (existing) ---
        "Basic_avg_degree_norm","Basic_degree_entropy_norm",
        "Connectivity_diameter_norm","Connectivity_radius_norm","Wiener_mean_distance",
        "Motif_triangles_per_Cn3","Motif_4_cycles_per_Cn4","Motif_4_cliques_per_Cn4",
        "Motif_wedges_per_max",
        "Motif_induced_K1_3_per_Cn4","Motif_induced_P4_per_Cn4","Motif_induced_C4_per_Cn4",
        "Motif_induced_TailedTriangle_per_Cn4","Motif_induced_Diamond_per_Cn4","Motif_induced_K4_per_Cn4",
        "Robust_articulation_points_per_node","Robust_bridge_count_per_edge","Connectivity_num_components_per_node",
        "Centrality_closeness_mean_norm","Centrality_closeness_max_norm",
        "Spectral_algebraic_connectivity_over_avgdeg","Spectral_spectral_gap_rel",
        "Spectral_laplacian_heat_trace_t0.1_per_node","Spectral_laplacian_heat_trace_t1.0_per_node","Spectral_laplacian_heat_trace_t5.0_per_node",
        "Adjacency_energy_per_node","Adjacency_energy_over_fro","Adjacency_estrada_per_node","log_Adjacency_estrada_per_node",
        "Adjacency_moment_2_over_avgdeg","Adjacency_moment_3_over_avgdeg3","Adjacency_moment_4_over_avgdeg4",
        "Planarity_num_faces_over_upperbound","Planarity_face_size_mean_norm",
        "Symmetry_num_orbits_per_node","Symmetry_orbit_size_max_per_node","Symmetry_aut_size_log_over_log_nfact",
        "TDA_H0_count_per_node","TDA_H1_count_per_node",
        "TDA_H0_total_persistence_over_diam","TDA_H0_mean_persistence_over_diam","TDA_H0_max_persistence_over_diam",
        "TDA_H0_mean_birth_over_diam","TDA_H0_mean_death_over_diam",
        "TDA_H1_total_persistence_over_diam","TDA_H1_mean_persistence_over_diam","TDA_H1_max_persistence_over_diam",
        "TDA_H1_mean_birth_over_diam","TDA_H1_mean_death_over_diam",
        "TDA_Betti0_at_q25_per_node","TDA_Betti0_at_q50_per_node","TDA_Betti0_at_q75_per_node",
        "TDA_Betti1_at_q25_per_node","TDA_Betti1_at_q50_per_node","TDA_Betti1_at_q75_per_node",

        # --- NEW: extra coverage features ---
        "Assortativity_degree",
        "Clustering_mean","Clustering_q10","Clustering_q50","Clustering_q90",
        "Clustering_frac_zero","Clustering_frac_one",
        "Degree_gini",
        "Eff_diameter_p90","Ecc_mean","Ecc_q90",
        "Comm_modularity","Comm_count","Comm_size_max","Comm_size_gini","Comm_internal_edge_frac",
        "Motif_triangle_edge_incidence_median","Motif_triangle_edge_incidence_q90",
        "Motif_triangle_edge_frac_zero","Motif_triangle_edge_frac_ge2",
        "NetLSD_mean","NetLSD_std","NetLSD_q10","NetLSD_q90",
    ]

    if G.number_of_nodes() == 0:
        return {k: np.nan for k in base_keys}

    feats = {}
    n = G.number_of_nodes()
    m = G.number_of_edges()

    # I. Basic ----------------------------------------------------------------
    degs = [d for _, d in G.degree()]
    dh   = Counter(degs)
    feats.update({
        "Basic_num_nodes"          : n,
        "Basic_num_edges"          : m,
        "Basic_min_degree"         : min(degs) if degs else np.nan,
        "Basic_max_degree"         : max(degs) if degs else np.nan,
        "Basic_avg_degree"         : float(np.mean(degs)) if degs else np.nan,
        "Basic_degree_std"         : float(np.std(degs))  if degs else np.nan,
        "Basic_degree_skew"        : float(skew(degs))    if len(degs) > 2 else np.nan,
        "Basic_density"            : nx.density(G),
        "Basic_edge_to_node_ratio" : m / n if n else np.nan,
        "Basic_degree_entropy"     : shannon_entropy(dh),
    })

    # --- NEW: degree/local extras -------------------------------------------
    try:
        feats["Assortativity_degree"] = float(nx.degree_pearson_correlation_coefficient(G))
    except Exception:
        feats["Assortativity_degree"] = np.nan

    try:
        cc = nx.clustering(G)
        cc_vals = list(cc.values())
    except Exception:
        cc_vals = []
    feats["Clustering_mean"] = float(np.mean(cc_vals)) if cc_vals else np.nan
    feats["Clustering_q10"]  = _q(cc_vals, 0.10)
    feats["Clustering_q50"]  = _q(cc_vals, 0.50)
    feats["Clustering_q90"]  = _q(cc_vals, 0.90)
    feats["Clustering_frac_zero"] = _frac(cc_vals, lambda a: np.isclose(a, 0.0))
    feats["Clustering_frac_one"]  = _frac(cc_vals, lambda a: np.isclose(a, 1.0))
    feats["Degree_gini"] = gini(degs)

    # II. Connectivity -------------------------------------------------------
    feats.update({
        "Connectivity_is_connected"             : try_or_nan(nx.is_connected, G),
        "Connectivity_num_components"           : try_or_nan(nx.number_connected_components, G),
        "Connectivity_diameter"                 : try_or_nan(nx.diameter, G),
        "Connectivity_radius"                   : try_or_nan(nx.radius, G),
        "Connectivity_avg_shortest_path_length" : try_or_nan(nx.average_shortest_path_length, G),
        "Connectivity_wiener_index"             : try_or_nan(lambda g: nx.wiener_index(g), G),
    })

    # --- NEW: distances (sampled if big) ------------------------------------
    eff_p90, ecc_mean, ecc_p90 = _distance_stats_sampled(G)
    feats["Eff_diameter_p90"] = eff_p90
    feats["Ecc_mean"] = ecc_mean
    feats["Ecc_q90"]  = ecc_p90

    # III. Centrality --------------------------------------------------------
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        bc = try_or_nan(nx.betweenness_centrality, G, normalized=True)
        cc = try_or_nan(nx.closeness_centrality,  G)
        try:
            ec = nx.eigenvector_centrality_numpy(G)
        except Exception:
            ec = np.nan

    def stats(d):
        if isinstance(d, dict) and d:
            vals = list(d.values())
            return {"mean": np.mean(vals), "max": np.max(vals),
                    "std": np.std(vals),   "skew": skew(vals) if len(vals) > 2 else np.nan}
        return dict.fromkeys(("mean", "max", "std", "skew"), np.nan)

    feats.update({f"Centrality_betweenness_{k}": v for k, v in stats(bc).items()})
    feats.update({f"Centrality_closeness_{k}":  v for k, v in stats(cc).items()})
    feats.update({f"Centrality_eigenvector_{k}": v for k, v in stats(ec).items()})

    # IV. Core structure -----------------------------------------------------
    core_numbers = try_or_nan(nx.core_number, G)
    if isinstance(core_numbers, dict) and core_numbers:
        core_vals = list(core_numbers.values())
        feats["Core_max_core_index"]  = max(core_vals)
        feats["Core_core_index_mean"] = float(np.mean(core_vals))
    else:
        feats["Core_max_core_index"]  = np.nan
        feats["Core_core_index_mean"] = np.nan

    # V. Robustness ----------------------------------------------------------
    feats["Robust_articulation_points"] = try_or_nan(lambda g: len(list(nx.articulation_points(g))), G)
    feats["Robust_bridge_count"]        = try_or_nan(lambda g: len(list(nx.bridges(g))),            G)

    # VI. Cycle extras (existing) -------------------------------------------
    cbasis  = nx.cycle_basis(G)
    feats["Cycle_num_cycles_len_5"] = sum(1 for c in cbasis if len(c) == 5)
    feats["Cycle_num_cycles_len_6"] = sum(1 for c in cbasis if len(c) == 6)

    # VII. Spectral & Kirchhoff (existing) ----------------------------------
    leigs = None
    try:
        L     = nx.laplacian_matrix(G).todense()
        leigs = np.sort(np.linalg.eigvalsh(L))
        feats["Spectral_algebraic_connectivity"] = leigs[1]           if len(leigs) > 1 else np.nan
        feats["Spectral_spectral_gap"]           = leigs[1]-leigs[0]  if len(leigs) > 1 else np.nan
        feats["Spectral_laplacian_mean"]         = float(np.mean(leigs))
        feats["Spectral_laplacian_std"]          = float(np.std(leigs))
        feats["Spectral_laplacian_skew"]         = float(skew(leigs)) if len(leigs) > 2 else np.nan
        pad = np.full(10, np.nan)
        pad[:min(10, len(leigs))] = leigs[:10]
        feats.update({f"Spectral_lap_eig_{i}": float(pad[i]) for i in range(10)})
        nonzero = leigs[leigs > 1e-12]
        feats["Kirchhoff_index"] = float(n * np.sum(1.0 / nonzero)) if len(nonzero) >= 1 else np.nan
    except Exception:
        feats.update({
            "Spectral_algebraic_connectivity": np.nan,
            "Spectral_spectral_gap":           np.nan,
            "Spectral_laplacian_mean":         np.nan,
            "Spectral_laplacian_std":          np.nan,
            "Spectral_laplacian_skew":         np.nan,
            **{f"Spectral_lap_eig_{i}": np.nan for i in range(10)},
            "Kirchhoff_index": np.nan,
        })

    # --- NEW: NetLSD summary from Laplacian spectrum ------------------------
    if leigs is not None and len(leigs) > 0 and not np.any(np.isnan(leigs)):
        ts = np.logspace(-2, 2, 32)  # 0.01 ... 100
        heat = np.exp(-np.outer(ts, leigs)).sum(axis=1)  # trace(e^{-tL})
        feats["NetLSD_mean"] = float(np.mean(heat))
        feats["NetLSD_std"]  = float(np.std(heat))
        feats["NetLSD_q10"]  = float(np.quantile(heat, 0.10))
        feats["NetLSD_q90"]  = float(np.quantile(heat, 0.90))
    else:
        feats["NetLSD_mean"] = np.nan
        feats["NetLSD_std"]  = np.nan
        feats["NetLSD_q10"]  = np.nan
        feats["NetLSD_q90"]  = np.nan

    # VIII. Planarity  -------------------------------------------------------
    try:
        planar, emb = nx.check_planarity(G)
        if planar:
            face_list = extract_faces(emb)
            f_sizes = [len(face) for face in face_list]
            feats["Planarity_num_faces"]      = float(len(face_list))
            feats["Planarity_face_size_mean"] = float(np.mean(f_sizes)) if f_sizes else np.nan
            feats["Planarity_face_size_max"]  = float(np.max(f_sizes))  if f_sizes else np.nan
        else:
            feats["Planarity_num_faces"]      = np.nan
            feats["Planarity_face_size_mean"] = np.nan
            feats["Planarity_face_size_max"]  = np.nan
    except Exception:
        feats["Planarity_num_faces"]      = np.nan
        feats["Planarity_face_size_mean"] = np.nan
        feats["Planarity_face_size_max"]  = np.nan

    # IX. Symmetry -----------------------------------------------------------
    try:
        GM       = nx.algorithms.isomorphism.GraphMatcher(G, G)
        aut_size = len(list(GM.isomorphisms_iter()))
        orbits   = compute_node_orbits(G)
        feats["Symmetry_automorphism_group_order"] = float(aut_size)
        feats["Symmetry_num_orbits"]               = float(len(orbits))
        feats["Symmetry_orbit_size_max"]           = float(max(len(o) for o in orbits)) if orbits else np.nan
    except Exception:
        feats["Symmetry_automorphism_group_order"] = np.nan
        feats["Symmetry_num_orbits"]               = np.nan
        feats["Symmetry_orbit_size_max"]           = np.nan

    # --------------------------- NEW: community features ---------------------
    feats.update(_community_features(G))

    # --------------------------- NEW: motifs (raw) --------------------------
    A = _adjacency_matrix(G)
    tri = _triangle_count(G, A=A)
    feats["Motif_triangles"]   = tri
    feats["Motif_wedges"]      = _wedge_count(G, degs=degs, tri=tri)
    feats["Motif_4_cycles"]    = _four_cycle_count(G, A=A)
    feats["Motif_4_cliques"]   = _four_clique_count(G)
    m_mean, m_std = _edge_triangle_incidence_stats(G)
    feats["Motif_triangle_edge_incidence_mean"] = m_mean
    feats["Motif_triangle_edge_incidence_std"]  = m_std
    feats["Motif_square_clustering_proxy"]      = _square_clustering_proxy(G, A=A)

    # --- NEW: embeddedness distribution details -----------------------------
    emb_list = [len(set(G[u]) & set(G[v])) for u, v in G.edges()]
    feats["Motif_triangle_edge_incidence_median"] = _q(emb_list, 0.50)
    feats["Motif_triangle_edge_incidence_q90"]    = _q(emb_list, 0.90)
    feats["Motif_triangle_edge_frac_zero"]        = _frac(emb_list, lambda a: np.isclose(a, 0.0))
    feats["Motif_triangle_edge_frac_ge2"]         = _frac(emb_list, lambda a: a >= 2)

    ind4 = _induced4_counts(G)
    for k, v in ind4.items():
        feats[f"Motif_induced_{k}"] = float(v)

    # frac of 4-sets that are connected (any of the 6 induced classes)
    total_4sets = safe_comb(n, 4) if n >= 4 else np.nan
    feats["Motif_induced_connected_per_4set"] = float(sum(ind4.values()) / total_4sets) if (total_4sets is not None and total_4sets==total_4sets and total_4sets) else np.nan

    # --------- NEW: adjacency spectral/energy + Laplacian heat traces -------
    energy, estrada, m2, m3, m4, _ = _adjacency_spectrum_features(A)
    feats["Adjacency_energy"]        = energy
    feats["Adjacency_estrada_index"] = estrada
    feats["Adjacency_moment_2"]      = m2
    feats["Adjacency_moment_3"]      = m3
    feats["Adjacency_moment_4"]      = m4
    feats.update(_laplacian_heat_traces(leigs, ts=(0.1, 1.0, 5.0)))

    # ----------------------- NEW: TDA via VR (extended) ---------------------
    feats.update(_tda_vr_h_summary_from_spdm(G))

    # ----------------------------- Normalizations ---------------------------
    n_f = float(n); m_f = float(m)
    avgdeg = feats["Basic_avg_degree"] if feats["Basic_avg_degree"]==feats["Basic_avg_degree"] else np.nan

    # Basic / distances
    feats["Basic_avg_degree_norm"]       = feats["Basic_avg_degree"] / max(n_f-1, 1) if n_f==n_f else np.nan
    feats["Basic_degree_entropy_norm"]   = feats["Basic_degree_entropy"] / np.log2(max(n_f-1, 2)) if n_f==n_f else np.nan
    feats["Connectivity_diameter_norm"]  = feats["Connectivity_diameter"] / max(n_f-1, 1) if feats["Connectivity_diameter"]==feats["Connectivity_diameter"] and n_f==n_f else np.nan
    feats["Connectivity_radius_norm"]    = feats["Connectivity_radius"] / max(n_f-1, 1) if feats["Connectivity_radius"]==feats["Connectivity_radius"] and n_f==n_f else np.nan
    pairs = n_f*(n_f-1)/2.0 if n_f==n_f else np.nan
    feats["Wiener_mean_distance"] = feats["Connectivity_wiener_index"]/pairs if feats["Connectivity_wiener_index"]==feats["Connectivity_wiener_index"] and pairs and pairs==pairs else np.nan

    # Motifs
    Cn3 = safe_comb(n,3); Cn4 = safe_comb(n,4)
    feats["Motif_triangles_per_Cn3"]     = feats["Motif_triangles"]/Cn3 if Cn3 and Cn3==Cn3 else np.nan
    feats["Motif_4_cycles_per_Cn4"]      = feats["Motif_4_cycles"]/Cn4  if Cn4 and Cn4==Cn4 else np.nan
    feats["Motif_4_cliques_per_Cn4"]     = feats["Motif_4_cliques"]/Cn4 if Cn4 and Cn4==Cn4 else np.nan
    max_wedges = n_f*((n_f-1)*(n_f-2)/2.0) if n_f==n_f and n>=3 else np.nan
    feats["Motif_wedges_per_max"]        = feats["Motif_wedges"]/max_wedges if max_wedges and max_wedges==max_wedges else np.nan

    for key in ["K1_3","P4","C4","TailedTriangle","Diamond","K4"]:
        raw = feats.get(f"Motif_induced_{key}", np.nan)
        feats[f"Motif_induced_{key}_per_Cn4"] = raw/Cn4 if Cn4 and Cn4==Cn4 else np.nan

    # Robustness
    feats["Robust_articulation_points_per_node"] = feats["Robust_articulation_points"]/n_f if n_f and n_f==n_f else np.nan
    feats["Robust_bridge_count_per_edge"]        = feats["Robust_bridge_count"]/m_f if m_f and m_f==m_f else np.nan
    feats["Connectivity_num_components_per_node"]= feats["Connectivity_num_components"]/n_f if n_f and n_f==n_f else np.nan

    # Centralities (closeness normalization by (n-1))
    feats["Centrality_closeness_mean_norm"] = feats["Centrality_closeness_mean"]*(n_f-1) if feats["Centrality_closeness_mean"]==feats["Centrality_closeness_mean"] and n_f==n_f else np.nan
    feats["Centrality_closeness_max_norm"]  = feats["Centrality_closeness_max"] *(n_f-1) if feats["Centrality_closeness_max"] ==feats["Centrality_closeness_max"]  and n_f==n_f else np.nan

    # Spectral
    feats["Spectral_algebraic_connectivity_over_avgdeg"] = feats["Spectral_algebraic_connectivity"]/max(avgdeg,1e-9) if feats["Spectral_algebraic_connectivity"]==feats["Spectral_algebraic_connectivity"] and avgdeg==avgdeg else np.nan
    feats["Spectral_spectral_gap_rel"] = feats["Spectral_spectral_gap"]/max(feats["Spectral_laplacian_mean"],1e-9) if feats["Spectral_spectral_gap"]==feats["Spectral_spectral_gap"] and feats["Spectral_laplacian_mean"]==feats["Spectral_laplacian_mean"] else np.nan

    for t in ("0.1","1.0","5.0"):
        col = f"Spectral_laplacian_heat_trace_t{t}"
        feats[f"{col}_per_node"] = feats[col]/n_f if col in feats and feats[col]==feats[col] and n_f==n_f else np.nan

    feats["Adjacency_energy_per_node"] = feats["Adjacency_energy"]/n_f if feats["Adjacency_energy"]==feats["Adjacency_energy"] and n_f==n_f else np.nan
    feats["Adjacency_energy_over_fro"] = feats["Adjacency_energy"]/math.sqrt(2.0*max(m_f,1e-12)) if feats["Adjacency_energy"]==feats["Adjacency_energy"] and m_f==m_f else np.nan
    feats["Adjacency_estrada_per_node"]= feats["Adjacency_estrada_index"]/n_f if feats["Adjacency_estrada_index"]==feats["Adjacency_estrada_index"] and n_f==n_f else np.nan
    feats["log_Adjacency_estrada_per_node"] = np.log(max(feats["Adjacency_estrada_index"],1.0))/n_f if feats["Adjacency_estrada_index"]==feats["Adjacency_estrada_index"] and n_f==n_f else np.nan

    feats["Adjacency_moment_2_over_avgdeg"]  = feats["Adjacency_moment_2"] / max(avgdeg,1e-9) if feats["Adjacency_moment_2"]==feats["Adjacency_moment_2"] and avgdeg==avgdeg else np.nan
    feats["Adjacency_moment_3_over_avgdeg3"] = feats["Adjacency_moment_3"] / max(avgdeg**3,1e-9) if feats["Adjacency_moment_3"]==feats["Adjacency_moment_3"] and avgdeg==avgdeg else np.nan
    feats["Adjacency_moment_4_over_avgdeg4"] = feats["Adjacency_moment_4"] / max(avgdeg**4,1e-9) if feats["Adjacency_moment_4"]==feats["Adjacency_moment_4"] and avgdeg==avgdeg else np.nan

    # Planarity (normalized)
    upper = 2*float(n) - 4.0 if n >= 3 else np.nan  # connected planar simple graphs
    faces = feats.get("Planarity_num_faces", np.nan)
    feats["Planarity_num_faces_over_upperbound"] = faces/upper if (faces==faces and upper==upper and upper) else np.nan
    feats["Planarity_face_size_mean_norm"] = feats["Planarity_face_size_mean"]/float(n) if feats["Planarity_face_size_mean"]==feats["Planarity_face_size_mean"] else np.nan

    # Symmetry
    try:
        aut = feats["Symmetry_automorphism_group_order"]
        log_aut  = np.log(max(aut,1.0)) if aut==aut else np.nan
        log_nfact = math.lgamma(float(n)+1.0)
        feats["Symmetry_aut_size_log_over_log_nfact"] = (log_aut / max(log_nfact,1e-9)) if log_aut==log_aut and log_nfact==log_nfact else np.nan
    except Exception:
        feats["Symmetry_aut_size_log_over_log_nfact"] = np.nan

    feats["Symmetry_num_orbits_per_node"]    = feats["Symmetry_num_orbits"]/float(n) if feats["Symmetry_num_orbits"]==feats["Symmetry_num_orbits"] else np.nan
    feats["Symmetry_orbit_size_max_per_node"]= feats["Symmetry_orbit_size_max"]/float(n) if feats["Symmetry_orbit_size_max"]==feats["Symmetry_orbit_size_max"] else np.nan

    # TDA
    for H in ["H0","H1"]:
        cnt = feats.get(f"TDA_{H}_count", np.nan)
        feats[f"TDA_{H}_count_per_node"] = cnt/float(n) if cnt==cnt else np.nan
        for stat in ["total_persistence","mean_persistence","max_persistence","mean_birth","mean_death"]:
            col = f"TDA_{H}_{stat}"
            val = feats.get(col, np.nan)
            feats[col + "_over_diam"] = val / max(float(n)-1.0,1.0) if val==val else np.nan

    for q in ["q25","q50","q75"]:
        c0 = f"TDA_Betti0_at_{q}"; c1 = f"TDA_Betti1_at_{q}"
        if c0 in feats and feats[c0]==feats[c0]:
            feats[c0 + "_per_node"] = feats[c0]/float(n)
        if c1 in feats and feats[c1]==feats[c1]:
            feats[c1 + "_per_node"] = feats[c1]/float(n)

    # Ensure all keys present
    for k in base_keys:
        feats.setdefault(k, np.nan)
    return feats

# ----------------------------- Batch/parallel ------------------------------ #
def _safe_extract(edge_list):
    try:
        return extract_features(edge_list)
    except Exception as e:
        # Return a sparse marker row with NaNs
        d = {"__error__": str(e)}
        return {**{k: np.nan for k in extract_features([]).keys()}, **d}

def compute_batch(df_edges: pd.Series, workers: int = 0, backend: str = "loky"):
    items = df_edges.tolist()

    if workers is None or workers <= 0:
        # serial
        out = []
        for edges in tqdm(items, desc="Extracting graph features"):
            out.append(_safe_extract(edges))
        return pd.DataFrame(out)

    if not _HAS_JOBLIB:
        # fallback to serial if joblib not available
        out = []
        for edges in tqdm(items, desc="Extracting graph features"):
            out.append(_safe_extract(edges))
        return pd.DataFrame(out)

    # joblib path
    iterator = (delayed(_safe_extract)(edges) for edges in items)
    if _HAS_TQDM_JOBLIB:
        with tqdm_joblib(tqdm(total=len(items), desc="Extracting graph features (parallel)")):
            results = Parallel(n_jobs=workers, backend=backend)(iterator)
    else:
        # This gives a schedule-based progress, still useful
        results = Parallel(n_jobs=workers, backend=backend)(
            tqdm(list(iterator), total=len(items), desc="Scheduling tasks")
        )
    return pd.DataFrame(results)

# ---------------------------------- CLI ------------------------------------ #
def parse_args():
    p = argparse.ArgumentParser(description="Generate graph feature tables from EDGES CSV.")
    p.add_argument("--input", required=True, help="Input CSV with EDGES column.")
    p.add_argument("--output", required=True,
                   help="Output CSV path. If it contains {batch}, {start}, or {end}, batched files are written.")
    p.add_argument("--batch-size", type=int, default=0,
                   help="Number of rows per batch. 0 means process all at once to a single file.")
    p.add_argument("--workers", type=int, default=0,
                   help="Number of parallel workers (joblib). 0 or negative = no parallelism.")
    p.add_argument("--backend", choices=["loky", "threading"], default="loky",
                   help="Parallel backend (loky=processes, threading=threads).")
    p.add_argument("--seed", type=int, default=42, help="Random seed for sampling distances.")
    return p.parse_args()

def parse_edges_column(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    def _parse(x):
        if isinstance(x, (list, tuple)):
            return [tuple(e) for e in x]
        if isinstance(x, str):
            obj = ast.literal_eval(x)
            if isinstance(obj, (list, tuple)):
                return [tuple(e) for e in obj]
        # Fallback: empty graph if unparsable
        return []
    df["EDGES"] = df["EDGES"].apply(_parse)
    return df

def write_output(df_out: pd.DataFrame, path: str, append: bool, header: bool):
    os.makedirs(os.path.dirname(os.path.abspath(path)), exist_ok=True)
    df_out.to_csv(path, index=False, mode=("a" if append else "w"), header=header)
    print(f"✓ Feature table saved → {path}")


def main(inp, out):


    # load your CSV
    df = pd.read_csv(inp)
    df = parse_edges_column(df)

    # compute features (you can set workers=0 for serial, or >0 for parallel with joblib)
    feat_df = compute_batch(df["EDGES"], workers=4)

    # keep COEFFICIENTS if present
    keep_cols = [c for c in ("COEFFICIENTS",) if c in df.columns]
    final_df = pd.concat([df[keep_cols].reset_index(drop=True), feat_df.reset_index(drop=True)], axis=1)

    # save or inspect
    final_df.to_csv(out, index=False)
    final_df.head()


In [96]:
import warnings
warnings.filterwarnings("ignore")

import warnings

# ignore all the warnings you listed
warnings.filterwarnings("ignore", category=RuntimeWarning, message=".*Precision loss.*")

import warnings
from scipy.stats import ConstantInputWarning

# Silence "Precision loss..." RuntimeWarnings
warnings.filterwarnings("ignore", category=RuntimeWarning, message=".*Precision loss.*")

# Silence ConstantInputWarning from scipy/networkx
warnings.filterwarnings("ignore", category=ConstantInputWarning)

In [97]:
main('/Users/rezadoobary/Documents/ML-correlator/Graph_Edge_Data/den_graph_data_7.csv',\
     '/Users/rezadoobary/Documents/ML-correlator/Tree classifier for graphs/mixed_loops/features_tabular/7loopfeats_enhanced.csv',
     )

Scheduling tasks:  46%|████▋     | 76/164 [00:01<00:01, 82.46it/s]/var/folders/4s/7f5zf8596_526pnhktn077f40000gn/T/ipykernel_19556/1087343297.py:568: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
/opt/miniconda3/envs/torch-env/lib/python3.10/site-packages/networkx/algorithms/assortativity/correlation.py:140: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  return sp.stats.pearsonr(x, y)[0]
/var/folders/4s/7f5zf8596_526pnhktn077f40000gn/T/ipykernel_19556/1087343297.py:623: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
Scheduling tasks: 100%|██████████| 164/164 [00:01<00:00, 85.51it/s] 
/var/folders/4s/7f5zf8596_526pnhktn077f40000gn/T/ipykernel_19556/1087343297.py:568: RuntimeWarning: Precision l

,COEFFICIENTS,Basic_num_nodes,Basic_num_edges,Basic_min_degree,Basic_max_degree,Basic_avg_degree,Basic_degree_std,Basic_degree_skew,Basic_density,Basic_edge_to_node_ratio,...,TDA_H1_mean_persistence_over_diam,TDA_H1_max_persistence_over_diam,TDA_H1_mean_birth_over_diam,TDA_H1_mean_death_over_diam,TDA_Betti0_at_q25_per_node,TDA_Betti1_at_q25_per_node,TDA_Betti0_at_q50_per_node,TDA_Betti1_at_q50_per_node,TDA_Betti0_at_q75_per_node,TDA_Betti1_at_q75_per_node
0,1,11,27,4,6,4.909091,0.514259,-0.132583,0.490909,2.454545,...,0.0,0.0,0.0,0.0,0.090909,0.000000,0.090909,0.0,0.090909,0.0
1,1,11,26,4,5,4.727273,0.445362,-1.020621,0.472727,2.363636,...,0.0,0.0,0.0,0.0,0.090909,0.000000,0.090909,0.0,0.090909,0.0
2,1,11,27,4,7,4.909091,0.899954,0.927691,0.490909,2.454545,...,0.0,0.0,0.0,0.0,0.090909,0.000000,0.090909,0.0,0.090909,0.0
3,1,11,26,4,6,4.727273,0.749656,0.492205,0.472727,2.363636,...,0.0,0.0,0.0,0.0,0.090909,0.000000,0.090909,0.0,0.090909,0.0
4,1,11,26,4,6,4.727273,0.749656,0.492205,0.472727,2.363636,...,0.0,0.0,0.0,0.0,0.090909,0.000000,0.090909,0.0,0.090909,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,0,11,25,4,6,4.545455,0.782030,0.989676,0.454545,2.272727,...,0.1,0.1,0.1,0.2,0.090909,0.090909,0.090909,0.0,0.090909,0.0
160,0,11,26,4,7,4.727273,1.135454,1.293343,0.472727,2.363636,...,0.0,0.0,0.0,0.0,0.090909,0.000000,0.090909,0.0,0.090909,0.0
161,0,11,24,4,6,4.363636,0.771389,1.649916,0.436364,2.181818,...,0.1,0.1,0.1,0.2,0.090909,0.181818,0.090909,0.0,0.090909,0.0
162,0,11,25,4,6,4.545455,0.655555,0.800048,0.454545,2.272727,...,0.1,0.1,0.1,0.2,0.090909,0.090909,0.090909,0.0,0.090909,0.0


In [98]:
main('/Users/rezadoobary/Documents/ML-correlator/Graph_Edge_Data/den_graph_data_8.csv',\
     '/Users/rezadoobary/Documents/ML-correlator/Tree classifier for graphs/mixed_loops/features_tabular/8loopfeats_enhanced.csv',
     )

Scheduling tasks:   0%|          | 0/1432 [00:00<?, ?it/s]/var/folders/4s/7f5zf8596_526pnhktn077f40000gn/T/ipykernel_19556/1087343297.py:568: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
/opt/miniconda3/envs/torch-env/lib/python3.10/site-packages/networkx/algorithms/assortativity/correlation.py:140: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  return sp.stats.pearsonr(x, y)[0]
/var/folders/4s/7f5zf8596_526pnhktn077f40000gn/T/ipykernel_19556/1087343297.py:623: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
Scheduling tasks:  11%|█         | 152/1432 [00:00<00:05, 229.38it/s]/var/folders/4s/7f5zf8596_526pnhktn077f40000gn/T/ipykernel_19556/1087343297.py:568: RuntimeWarning: Precision loss occu

,COEFFICIENTS,Basic_num_nodes,Basic_num_edges,Basic_min_degree,Basic_max_degree,Basic_avg_degree,Basic_degree_std,Basic_degree_skew,Basic_density,Basic_edge_to_node_ratio,...,TDA_H1_mean_persistence_over_diam,TDA_H1_max_persistence_over_diam,TDA_H1_mean_birth_over_diam,TDA_H1_mean_death_over_diam,TDA_Betti0_at_q25_per_node,TDA_Betti1_at_q25_per_node,TDA_Betti0_at_q50_per_node,TDA_Betti1_at_q50_per_node,TDA_Betti0_at_q75_per_node,TDA_Betti1_at_q75_per_node
0,1,12,30,4,6,5.000000,0.577350,0.000000,0.454545,2.500000,...,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.083333,0.0,0.083333,0.0
1,1,12,29,4,5,4.833333,0.372678,-1.788854,0.439394,2.416667,...,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.083333,0.0,0.083333,0.0
2,1,12,30,5,5,5.000000,0.000000,NaN,0.454545,2.500000,...,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.083333,0.0,0.083333,0.0
3,1,12,30,4,7,5.000000,0.816497,0.918559,0.454545,2.500000,...,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.083333,0.0,0.083333,0.0
4,1,12,29,4,6,4.833333,0.687184,0.228269,0.439394,2.416667,...,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.083333,0.0,0.083333,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1427,0,12,27,4,5,4.500000,0.500000,0.000000,0.409091,2.250000,...,0.090909,0.090909,0.090909,0.181818,0.083333,0.166667,0.083333,0.0,0.083333,0.0
1428,0,12,28,4,6,4.666667,0.745356,0.626099,0.424242,2.333333,...,0.090909,0.090909,0.090909,0.181818,0.083333,0.083333,0.083333,0.0,0.083333,0.0
1429,0,12,26,4,5,4.333333,0.471405,0.707107,0.393939,2.166667,...,0.090909,0.090909,0.090909,0.181818,0.083333,0.083333,0.083333,0.0,0.083333,0.0
1430,0,12,26,4,5,4.333333,0.471405,0.707107,0.393939,2.166667,...,0.090909,0.090909,0.090909,0.181818,0.083333,0.083333,0.083333,0.0,0.083333,0.0


In [102]:
main('/Users/rezadoobary/Documents/ML-correlator/Graph_Edge_Data/den_graph_data_9.csv',\
     '/Users/rezadoobary/Documents/ML-correlator/Tree classifier for graphs/mixed_loops/features_tabular/9loopfeats_enhanced.csv',
     )

Scheduling tasks:   3%|▎         | 356/13972 [00:01<00:58, 231.89it/s]/var/folders/4s/7f5zf8596_526pnhktn077f40000gn/T/ipykernel_19556/3650404104.py:562: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
/opt/miniconda3/envs/torch-env/lib/python3.10/site-packages/networkx/algorithms/assortativity/correlation.py:140: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  return sp.stats.pearsonr(x, y)[0]
Scheduling tasks:   5%|▌         | 708/13972 [00:03<00:57, 232.68it/s]/var/folders/4s/7f5zf8596_526pnhktn077f40000gn/T/ipykernel_19556/3650404104.py:562: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
/opt/miniconda3/envs/torch-env/lib/python3.10/site-packages/networkx/algorithms/assortativity/correlation.

In [69]:
df = pd.read_csv('/Users/rezadoobary/Documents/ML-correlator/Tree classifier for graphs/mixed_loops/features_tabular/9loopfeats_enhanced.csv')

In [74]:
from tqdm.notebook import tqdm

In [75]:
def main(df, out):
    if "EDGES" not in df.columns:
        raise ValueError("CSV must contain an 'EDGES' column")
    df["EDGES"] = df["EDGES"].apply(ast.literal_eval)

    tqdm.pandas(desc="Extracting graph features")
    feat_df = pd.DataFrame(df["EDGES"].progress_apply(extract_features).tolist())

    keep_cols = [c for c in ("COEFFICIENTS",) if c in df.columns]
    final_df  = pd.concat([df[keep_cols], feat_df], axis=1)

    final_df.to_csv(out, index=False)
    print(f"✓ Feature table saved → {out}")

temp = pd.read_csv('/Users/rezadoobary/Documents/ML-correlator/Graph_Edge_Data/den_graph_data_10.csv')
batch_size = 10_000
num_rows = len(temp)

for i in tqdm(range(0, num_rows, batch_size)):
    # Slice the DataFrame into batches
    temp_batch = temp.iloc[i:i+batch_size]

    # Batch index for naming
    batch_num = i // batch_size + 1

    # Call your function
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        main(
            temp_batch,
            f"/Users/rezadoobary/Documents/ML-correlator/Tree classifier for graphs/mixed_loops/features_tabular/10loopfeats_enhanced_{batch_num}.csv"
        )

  0%|          | 0/16 [00:00<?, ?it/s]

Extracting graph features:   0%|          | 0/10000 [00:00<?, ?it/s]

✓ Feature table saved → /Users/rezadoobary/Documents/ML-correlator/Tree classifier for graphs/mixed_loops/features_tabular/10loopfeats_enhanced_1.csv


Extracting graph features:   0%|          | 0/10000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [57]:
df = pd.read_csv("/Users/rezadoobary/Documents/ML-correlator/Tree classifier for graphs/mixed_loops/features_tabular/9loopfeats_enhanced.csv")

In [60]:
df['__error__'].iloc[0]

"'<=' not supported between instances of 'int' and 'NoneType'"

In [20]:
main('/Users/rezadoobary/Documents/ML-correlator/Graph_Edge_Data/den_graph_data_9.csv',\
     '/Users/rezadoobary/Documents/ML-correlator/Tree classifier for graphs/mixed_loops/features_tabular/9loopfeats_enhanced.csv')

Extracting graph features:   3%|▎         | 352/13972 [00:05<03:33, 63.85it/s]/var/folders/4s/7f5zf8596_526pnhktn077f40000gn/T/ipykernel_19556/2550374772.py:438: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  "Basic_degree_skew"        : float(skew(degs))    if len(degs) > 2 else np.nan,
/var/folders/4s/7f5zf8596_526pnhktn077f40000gn/T/ipykernel_19556/2550374772.py:468: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  "std": np.std(vals),   "skew": skew(vals) if len(vals) > 2 else np.nan}
Extracting graph features:   5%|▍         | 660/13972 [00:10<03:11, 69.37it/s]/var/folders/4s/7f5zf8596_526pnhktn077f40000gn/T/ipykernel_19556/2550374772.py:438: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation

✓ Feature table saved → /Users/rezadoobary/Documents/ML-correlator/Tree classifier for graphs/mixed_loops/features_tabular/9loopfeats_enhanced.csv


In [21]:
main('/Users/rezadoobary/Documents/ML-correlator/Graph_Edge_Data/den_graph_data_10.csv',\
     '/Users/rezadoobary/Documents/ML-correlator/Tree classifier for graphs/mixed_loops/features_tabular/10loopfeats_enhanced.csv')

Extracting graph features:   2%|▏         | 3006/153252 [00:54<49:53, 50.19it/s] /var/folders/4s/7f5zf8596_526pnhktn077f40000gn/T/ipykernel_19556/2550374772.py:438: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  "Basic_degree_skew"        : float(skew(degs))    if len(degs) > 2 else np.nan,
/var/folders/4s/7f5zf8596_526pnhktn077f40000gn/T/ipykernel_19556/2550374772.py:468: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  "std": np.std(vals),   "skew": skew(vals) if len(vals) > 2 else np.nan}
Extracting graph features:   2%|▏         | 3534/153252 [01:04<42:59, 58.05it/s]/var/folders/4s/7f5zf8596_526pnhktn077f40000gn/T/ipykernel_19556/2550374772.py:438: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancell

✓ Feature table saved → /Users/rezadoobary/Documents/ML-correlator/Tree classifier for graphs/mixed_loops/features_tabular/10loopfeats_enhanced.csv


In [42]:
from tqdm.notebook import tqdm

In [24]:
temp = pd.read_csv('/Users/rezadoobary/Documents/ML-correlator/Graph_Edge_Data/den_graph_data_11.csv')

In [43]:
def main(df, out):
    if "EDGES" not in df.columns:
        raise ValueError("CSV must contain an 'EDGES' column")
    df["EDGES"] = df["EDGES"].apply(ast.literal_eval)

    tqdm.pandas(desc="Extracting graph features")
    feat_df = pd.DataFrame(df["EDGES"].progress_apply(extract_features).tolist())

    keep_cols = [c for c in ("COEFFICIENTS",) if c in df.columns]
    final_df  = pd.concat([df[keep_cols], feat_df], axis=1)

    final_df.to_csv(out, index=False)
    print(f"✓ Feature table saved → {out}")

In [44]:
import warnings
warnings.filterwarnings("ignore")

In [45]:
batch_size = 100_000
num_rows = len(temp)

for i in tqdm(range(0, num_rows, batch_size)):
    # Slice the DataFrame into batches
    temp_batch = temp.iloc[i:i+batch_size]

    # Batch index for naming
    batch_num = i // batch_size + 1

    # Call your function
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        main(
            temp_batch,
            f"/Users/rezadoobary/Documents/ML-correlator/Tree classifier for graphs/mixed_loops/features_tabular/11loopfeats_enhanced_{batch_num}.csv"
        )

  0%|          | 0/17 [00:00<?, ?it/s]

Extracting graph features:   0%|          | 0/100000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [10]:
import pandas as pd
import ast
from collections import Counter

# ---- 2. helpers -------------------------------------------------------------
def to_edge_multiset(edge_col_str: str) -> Counter:
    """
    Convert the string "[ (1,2), (3,4), ... ]"  →  Counter({(1,2):1, (3,4):1, ...})
    Using Counter lets us treat the lists as multisets, so duplicate edges
    are handled correctly (e.g. two (8,9) entries remove two copies, not one).
    """
    # edge_col_str is a stringified Python list, so ast.literal_eval is safe
    edges = ast.literal_eval(edge_col_str)
    return Counter(tuple(e) for e in edges)     # ensure (immutable) tuples

def remove_num_from_den(row) -> list[tuple]:
    den = to_edge_multiset(row["DEN_EDGES"])
    num = to_edge_multiset(row["NUM_EDGES"])
    # Subtract multiplicities (Counter subtraction clips at zero automatically)
    remaining = den - num
    # Return a regular list for readability / downstream use
    return list(remaining.elements())


def generate_diff_graphs(input, out):
    df = pd.read_csv(input)          # adjust the filename/path as needed
    # ---- 3. apply it ------------------------------------------------------------
    df["EDGES"] = df.apply(remove_num_from_den, axis=1)

    # ---- 4. (optional) save or inspect -----------------------------------------
    df = df[["COEFFICIENTS", "EDGES"]]
    # df.to_csv("edges_minus_num.csv", index=False)
    tqdm.pandas(desc="Extracting graph features")
    feat_df = pd.DataFrame(df["EDGES"].progress_apply(extract_features).tolist())

    keep_cols = [c for c in ("COEFFICIENTS",) if c in df.columns]
    final_df  = pd.concat([df[keep_cols], feat_df], axis=1)

    final_df.to_csv(out, index=False)
    print(f"✓ Feature table saved → {out}")


In [11]:
generate_diff_graphs("/Users/rezadoobary/Documents/ML-correlator/Graph_Edge_Data/graph_data_5.csv",\
                      '/Users/rezadoobary/Documents/ML-correlator/Tree classifier for graphs/mixed_loops/features_tabular/5loopdifffeats.csv')

Extracting graph features:   0%|          | 0/7 [00:00<?, ?it/s]/var/folders/4s/7f5zf8596_526pnhktn077f40000gn/T/ipykernel_84618/3248177413.py:92: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  "Basic_degree_skew"        : float(skew(degs))    if len(degs) > 2 else np.nan,
/var/folders/4s/7f5zf8596_526pnhktn077f40000gn/T/ipykernel_84618/3248177413.py:122: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  "std": np.std(vals),   "skew": skew(vals) if len(vals) > 2 else np.nan}
Extracting graph features: 100%|██████████| 7/7 [00:00<00:00, 90.30it/s]

✓ Feature table saved → /Users/rezadoobary/Documents/ML-correlator/Tree classifier for graphs/mixed_loops/features_tabular/5loopdifffeats.csv


In [12]:
generate_diff_graphs("/Users/rezadoobary/Documents/ML-correlator/Graph_Edge_Data/graph_data_6.csv",\
                      '/Users/rezadoobary/Documents/ML-correlator/Tree classifier for graphs/mixed_loops/features_tabular/6loopdifffeats.csv')

Extracting graph features:   0%|          | 0/36 [00:00<?, ?it/s]/var/folders/4s/7f5zf8596_526pnhktn077f40000gn/T/ipykernel_84618/3248177413.py:92: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  "Basic_degree_skew"        : float(skew(degs))    if len(degs) > 2 else np.nan,
/var/folders/4s/7f5zf8596_526pnhktn077f40000gn/T/ipykernel_84618/3248177413.py:122: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  "std": np.std(vals),   "skew": skew(vals) if len(vals) > 2 else np.nan}
Extracting graph features:  47%|████▋     | 17/36 [00:00<00:00, 157.42it/s]/var/folders/4s/7f5zf8596_526pnhktn077f40000gn/T/ipykernel_84618/3248177413.py:92: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when

✓ Feature table saved → /Users/rezadoobary/Documents/ML-correlator/Tree classifier for graphs/mixed_loops/features_tabular/6loopdifffeats.csv


In [ ]:
generate_diff_graphs("/Users/rezadoobary/Documents/ML-correlator/Graph_Edge_Data/graph_data_7.csv",\
                      '/Users/rezadoobary/Documents/ML-correlator/Tree classifier for graphs/mixed_loops/features_tabular/7loopdifffeats.csv')

Extracting graph features:  47%|████▋     | 104/220 [00:00<00:00, 238.90it/s]/var/folders/4s/7f5zf8596_526pnhktn077f40000gn/T/ipykernel_84618/3248177413.py:92: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  "Basic_degree_skew"        : float(skew(degs))    if len(degs) > 2 else np.nan,
/var/folders/4s/7f5zf8596_526pnhktn077f40000gn/T/ipykernel_84618/3248177413.py:122: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  "std": np.std(vals),   "skew": skew(vals) if len(vals) > 2 else np.nan}
Extracting graph features:  71%|███████   | 156/220 [00:00<00:00, 223.61it/s]/var/folders/4s/7f5zf8596_526pnhktn077f40000gn/T/ipykernel_84618/3248177413.py:92: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. Th

✓ Feature table saved → /Users/rezadoobary/Documents/ML-correlator/Tree classifier for graphs/mixed_loops/features_tabular/7loopdifffeats.csv


In [14]:
generate_diff_graphs("/Users/rezadoobary/Documents/ML-correlator/Graph_Edge_Data/graph_data_8.csv",\
                      '/Users/rezadoobary/Documents/ML-correlator/Tree classifier for graphs/mixed_loops/features_tabular/8loopdifffeats.csv')

Extracting graph features:   1%|          | 23/2709 [00:00<00:12, 223.31it/s]/var/folders/4s/7f5zf8596_526pnhktn077f40000gn/T/ipykernel_84618/3248177413.py:92: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  "Basic_degree_skew"        : float(skew(degs))    if len(degs) > 2 else np.nan,
/var/folders/4s/7f5zf8596_526pnhktn077f40000gn/T/ipykernel_84618/3248177413.py:122: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  "std": np.std(vals),   "skew": skew(vals) if len(vals) > 2 else np.nan}
/var/folders/4s/7f5zf8596_526pnhktn077f40000gn/T/ipykernel_84618/3248177413.py:92: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  "B

✓ Feature table saved → /Users/rezadoobary/Documents/ML-correlator/Tree classifier for graphs/mixed_loops/features_tabular/8loopdifffeats.csv


In [15]:
import warnings
warnings.filterwarnings("ignore")

In [16]:
generate_diff_graphs("/Users/rezadoobary/Documents/ML-correlator/Graph_Edge_Data/graph_data_9.csv",\
                      '/Users/rezadoobary/Documents/ML-correlator/Tree classifier for graphs/mixed_loops/features_tabular/9loopdifffeats.csv')

Extracting graph features: 100%|██████████| 43017/43017 [02:49<00:00, 253.50it/s]


✓ Feature table saved → /Users/rezadoobary/Documents/ML-correlator/Tree classifier for graphs/mixed_loops/features_tabular/9loopdifffeats.csv


In [17]:
#  features space for the numerator + denominator sepeaerately

In [20]:
def main(inp, out):
    df = pd.read_csv(inp)
    if "EDGES" not in df.columns:
        raise ValueError("CSV must contain an 'EDGES' column")

    df["EDGES"] = df["EDGES"].apply(ast.literal_eval)

    tqdm.pandas(desc="Extracting graph features")
    feat_df = pd.DataFrame(df["EDGES"].progress_apply(extract_features).tolist())

    keep_cols = [c for c in ("COEFFICIENTS",) if c in df.columns]
    final_df  = pd.concat([df[keep_cols], feat_df], axis=1)

    final_df.to_csv(out, index=False)
    print(f"✓ Feature table saved → {out}")

In [50]:

def generate_individual_graph_features(input, out):
    df = pd.read_csv(input)
    df["DEN_EDGES"] = df["DEN_EDGES"].apply(ast.literal_eval)
    df["NUM_EDGES"] = df["NUM_EDGES"].apply(ast.literal_eval)

    tqdm.pandas(desc="Extracting graph features")
    feat_df = pd.DataFrame(df["DEN_EDGES"].progress_apply(extract_features).tolist())
    feat_df.columns = ['den_' + c for c in feat_df.columns]

    num_df = pd.DataFrame(df["NUM_EDGES"].progress_apply(extract_features).tolist())
    num_df.columns = ['num_' + c for c in num_df.columns]

    keep_cols = [c for c in ("COEFFICIENTS",) if c in df.columns]
    final_df  = pd.concat([df[keep_cols], feat_df, num_df], axis=1)

    final_df.to_csv(out, index=False)

In [52]:
generate_individual_graph_features("/Users/rezadoobary/Documents/ML-correlator/Graph_Edge_Data/graph_data_7.csv",\
                      '/Users/rezadoobary/Documents/ML-correlator/Tree classifier for graphs/mixed_loops/features_tabular/7loopmergedfeats.csv')

Extracting graph features: 100%|██████████| 220/220 [00:00<00:00, 313.08it/s]


In [ ]:
# takes too long to generate and does not seem to be worth it imo